Este notebook se basa en la adecuada limpieza de los datos segun el analisis exploratorio de los datos hecho en el notebook 002_EDA y se creara una nueva tabla en la misma base de datos creada en Postgres pero con los datos limpios. 

In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
print("Current directory before change:", os.getcwd())

try:
    
    os.chdir("../")
    print("Current directory after change:", os.getcwd())

except FileNotFoundError:
    print("""
        FileNotFoundError - The specified directory does not exist or you are already in the root.
        If the code already worked once, do not run it again.
    """)

Current directory before change: c:\Users\Administrador\Desktop\workshop_01\notebooks
Current directory after change: c:\Users\Administrador\Desktop\workshop_01


In [3]:
from db_access.connection import create_engine_connection
engine = create_engine_connection()

In [4]:
df = pd.read_sql("SELECT * FROM candidates", engine)

In [5]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print(df.columns)

Index(['first_name', 'last_name', 'email', 'application_date', 'country',
       'yoe', 'seniority', 'technology', 'code_challenge_score',
       'technical_interview_score'],
      dtype='object')


In [6]:
df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce')
df['application_date'] = df['application_date'].dt.strftime('%Y%m%d').astype(int)
df.head(1000)

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,20210226,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,20210909,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,20200414,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,20201001,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,20200520,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
995,Gladys,Blanda,maynard.kreiger63@hotmail.com,20200426,India,9,Trainee,System Administration,9,9
996,Johanna,Little,maiya45@hotmail.com,20200401,Aruba,8,Junior,Sales,6,10
997,Sister,Lindgren,erling.gaylord53@hotmail.com,20180612,Iceland,6,Intern,Mulesoft,2,0
998,Tyrese,Wilkinson,name68@gmail.com,20220206,Andorra,28,Intern,Security Compliance,0,9


In [7]:
df["technology"].unique()

array(['Data Engineer', 'Client Success', 'QA Manual',
       'Social Media Community Management', 'Adobe Experience Manager',
       'Sales', 'Mulesoft', 'DevOps', 'Development - CMS Backend',
       'Salesforce', 'System Administration', 'Security',
       'Game Development', 'Development - CMS Frontend',
       'Security Compliance', 'Development - Backend', 'Design',
       'Business Analytics / Project Management',
       'Development - Frontend', 'Development - FullStack',
       'Business Intelligence', 'Database Administration',
       'QA Automation', 'Technical Writing'], dtype=object)

In [8]:
technology_mapping = {
    'Data Engineer': 'Data & Analytics',
    'Business Analytics / Project Management': 'Data & Analytics',
    'Business Intelligence': 'Data & Analytics',

    'Client Success': 'Business & Strategy',
    'Sales': 'Business & Strategy',

    'QA Manual': 'Quality Assurance',
    'QA Automation': 'Quality Assurance',

    'Adobe Experience Manager': 'Software Engineering',
    'Development - CMS Backend': 'Software Engineering',
    'Development - CMS Frontend': 'Software Engineering',
    'Development - Backend': 'Software Engineering',
    'Development - Frontend': 'Software Engineering',
    'Development - FullStack': 'Software Engineering',
    'Game Development': 'Game Engineering',
  
    'Mulesoft': 'Automation & DevOps',
    'DevOps': 'Automation & DevOps',
    'Salesforce': 'Automation & DevOps',
  
    'System Administration': 'IT Infrastructure',
    'Database Administration': 'IT Infrastructure',

    'Security': 'Security & Compliance',
    'Security Compliance': 'Security & Compliance',
    'Design': 'Design & Creativity',
    'Social Media Community Management': 'Marketing & Communications',
    'Technical Writing': 'Content & Documentation'
}

df['technology_category'] = df['technology'].map(technology_mapping).fillna('Other')

In [9]:
df['hired'] = np.where((df['code_challenge_score'] >= 7) & (df['technical_interview_score'] >= 7), True, False)

In [ ]:
total_hired = df['hired'].sum()
print(f"Total de contratados: {total_hired}")


📊 Total de contratados: 6698


In [11]:
df.to_sql('candidates_eda', engine, if_exists='replace', index=False)
print("Carga exitosa en una nueva tabla ")

Carga exitosa en una nueva tabla 


- cambiar nombre de las columnas con _
- transfromar las fechas en enter
- crear una columna llamada hired que diga si esta contratado o no ( code_challenge_score y technical_interview_score deben ser ambos mayores o iguales a 7 para ser contratado)
- normalizar todas las columnas 
